# Tutorial 1 <br /> <br /> Solve Dantzig's Transport Problem using the *ix modeling platform* (ixmp4)

<img style="float: right; height: 80px;" src="_static/python.png">

## Aim and scope of the tutorial

This tutorial takes you through the steps to import the data for a very simple optimization model
and solve it using the **ixmp4**-GAMS interface.

We use Dantzig's transport problem, which is also used as the standard GAMS tutorial.
This problem finds a least cost shipping schedule that meets requirements at markets and supplies at factories.

If you are not familiar with GAMS, please take a minute to look at the [transport.gms](transport.gms) code.

For reference of the transport problem, see:
> Dantzig, G B, Chapter 3.3. In Linear Programming and Extensions.  
> Princeton University Press, Princeton, New Jersey, 1963.

> This formulation is described in detail in:  
> Rosenthal, R E, Chapter 2: A GAMS Tutorial.  
> In GAMS: A User's Guide. The Scientific Press, Redwood City, California, 1988.

> see http://www.gams.com/mccarl/trnsport.gms

## Tutorial outline

The steps in the tutorial are the following:

0. Launch an **ixmp4.Platform** instance and initialize a new **ixmp4.Run**
0. Define the **sets and parameters** in the scenario and save the data to the platform
0. Initialize **variables and equations** to import the solution from GAMS
0. Call GAMS to **solve the scenario** (export to GAMS input gdx, execute, read solution from output gdx)
0. Display the **solution** (variables and equation)

## Launching the *Platform* and initializing a new *Run*

A **Platform** is the connection to the database that holds all data and relevant additional information.

A **Run** is an object that holds all relevant information for one quantification of a scenario.  
A run is identified by a model name, a scenario name and a version number (assigned automatically).

You have to *register a new local database* before you can run the tutorial. 

Run the following in the command-line:
```
ixmp4 platforms add tutorial-test
```

You can then check if the database was successfully created by running
```
ixmp4 platforms list
```

After creating the database, you can connect to it via an **ixmp4.Platform** instance.

In [ ]:
import ixmp4

platform = ixmp4.Platform("tutorial-test")

Now, we initialize a new **ixmp4.Run** in the database. This is done by using the argument *version="new"*.

In [ ]:
run = platform.runs.create(model="transport problem", scenario="standard")

## Defining the *IndexSets*

An **IndexSet** defines a named list of elements. These IndexSets can be used for "indexed assignment" of parameters, variables and equations.  
In database-lingo, a column of a parameter can be "foreign-keyed" onto an IndexSet.

Below, we first show the data as they would be written in the GAMS tutorial ([transport.gms](transport.gms) in this folder).  

We now initialize these sets and assign the elements.

In [ ]:
i = run.optimization.indexsets.create("i")

In [ ]:
i.add(["seattle", "san-diego"])

We can display the elements of **IndexSet i** as a Python list.

In [ ]:
i.elements

For simplicity, the steps of creating an **IndexSet** and assigning elements can be done in one line.

In [ ]:
run.optimization.indexsets.create("j").add(["new-york", "chicago", "topeka"])

## Assigning the *Parameters*

Next, we define the parameters *capacity* and *demand*. The parameters are assigned on the IndexSets *i* and *j*, respectively.

In [ ]:
import pandas as pd

from ixmp4.core import Unit

# Only needed once for each mp
try:
    cases = platform.units.get("cases")
except Unit.NotFound:
    cases = platform.units.create("cases")

# capacity of plant i in cases
# add parameter data as a dict
a = run.optimization.parameters.create(name="a", constrained_to_indexsets=["i"])
a_data = {
    "i": ["seattle", "san-diego"],
    "values": [350, 600],
    "units": [cases.name, cases.name],
}
a.add(data=a_data)

# demand at market j in cases
# add parameter data as a pd.DataFrame
b = run.optimization.parameters.create("b", constrained_to_indexsets="j")
b_data = pd.DataFrame(
    [
        ["new-york", 325, cases.name],
        ["chicago", 300, cases.name],
        ["topeka", 275, cases.name],
    ],
    columns=["j", "values", "units"],
)
b.add(b_data)

In [ ]:
# And this is how e.g. b looks:
b.data

In [ ]:
try:
    km = platform.units.get("km")
except Unit.NotFound:
    km = platform.units.create("km")

# distance in thousands of miles
d = run.optimization.parameters.create("d", constrained_to_indexsets=["i", "j"])
# add more parameter data as dict
d_data = {
    "i": ["seattle", "seattle", "seattle", "san-diego"],
    "j": ["new-york", "chicago", "topeka", "new-york"],
    "values": [2.5, 1.7, 1.8, 2.5],
    "units": [km.name] * 4,
}
d.add(d_data)

# add other parameter data one by one
d.add({"i": ["san-diego"], "j": ["chicago"], "values": [1.8], "units": ["km"]})
d.add({"i": ["san-diego"], "j": ["topeka"], "values": [1.4], "units": ["km"]})

In [ ]:
# cost per case per 1000 miles

# TODO we could really use a units.get_or_create() function!
try:
    unit_cost_per_case = platform.units.get("USD/km")
except Unit.NotFound:
    unit_cost_per_case = platform.units.create("USD/km")

f = run.optimization.scalars.create(name="f", value=90, unit=unit_cost_per_case)

### Defining variables and equations in the scenario

The levels and marginals of these variables and equations will be imported to the scenario when reading the model solution.

In [ ]:
# initialize the decision variables and equations
z = run.optimization.variables.create("z")
x = run.optimization.variables.create("x", constrained_to_indexsets=["i", "j"])
supply = run.optimization.equations.create("supply", constrained_to_indexsets=["i"])
demand = run.optimization.equations.create("demand", constrained_to_indexsets=["j"])

### Solve the scenario

In this tutorial, we solve the tutorial using the ``highs`` solver in linopy. 

The ``create_dantzig_model()`` function is a convenience shortcut for setting up a linopy model correctly for the datzig scenario. Please see ``linopy_model.py`` for details.

The solution data are stored with the model object automatically. ``store_dantzig_solution()`` then stores them in the ixmp4 objects.

In [ ]:
from tutorial.transport.dantzig_model_linopy import (
    create_dantzig_model,
    read_dantzig_solution,
)

m = create_dantzig_model(run=run)
m.solve("highs")

read_dantzig_solution(model=m, run=run)

### Display and analyze the results

In [ ]:
# display the objective value of the solution
z.levels

In [ ]:
# display the quantities transported from canning plants to demand locations
x.data

In [ ]:
# display the quantities and marginals (shadow prices) of the demand balance constraints
demand.data

In [ ]:
# display the quantities and marginals (shadow prices) of the supply balance constraints
supply.data

In [ ]:
m.objective.coeffs.to_pandas().values